## Playing around with dataset, no useful DNN in this code

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import seaborn as sb
import matplotlib as mpl
import h5py
import pandas as pd 
import tables
from pandas.compat import StringIO, BytesIO 

mpl.rcParams['figure.dpi']= 250
pd.__file__

from torch.utils import data
#from hdf5dataset import HDF5Dataset

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
#open hdf5 file for reading
hdf_train = pd.HDFStore('train.h5',mode='r')
print(hdf_train.keys())
table_train = hdf_train.get('/table')

#hdf_val = pd.HDFStore('val.h5',mode='r')
#hdf_val.keys()
#table_val = hdf.get('/table')

#hdf_test = pd.HDFStore('test.h5',mode='r')
#hdf_test.keys()
#table_test = hdf.get('/table')


['/', '/table']


In [4]:
#type(table) DataFrame type
print(len(table_train.columns))
print(table_train.columns[0]) #E_0
print(table_train.columns[805]) #last column "is signal new" 1 = top, 0 = qcd
print(table_train.columns[15])

print(len(table_train))

806
E_0
is_signal_new
PZ_3
1211000


In [44]:
print(table_train.columns[-1])
X = table_train[table_train.columns[0:800]]
Y = table_train[table_train.columns[805]]

is_signal_new


In [42]:
Y

375    0
377    0
378    0
379    0
380    0
383    0
384    0
386    0
389    0
391    0
351    0
352    0
353    0
355    0
356    0
358    0
359    0
360    0
361    0
363    0
193    0
195    0
196    0
198    0
199    0
200    0
202    0
204    0
205    0
206    0
      ..
162    1
163    1
164    1
165    1
168    1
170    1
171    1
172    1
174    1
175    1
790    1
791    1
792    1
793    1
794    1
795    1
797    1
798    1
799    1
801    1
576    1
579    1
582    1
585    1
586    1
587    1
591    1
592    1
593    1
596    1
Name: is_signal_new, Length: 1211000, dtype: int64

In [9]:
table_train.shape


(1211000, 806)

In [102]:
#create t
train_tensor = torch.tensor(table_train.values)
#val_tensor = torch.tensor(table_val.values)
#test_tensor = torch.tensor(table_test.values)

sub_train_tensor = train_tensor[0:15]

#len(torch_tensor) 1211000

# another option, but this appears to be the same as using table...
# d = pd.read_hdf('train.h5','table')
# torch_tensor = torch.tensor(d.values)

In [78]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001


train_loader = torch.utils.data.DataLoader(dataset=train_tensor,
                                           batch_size=batch_size, 
                                           shuffle=True)

sub_train_loader = torch.utils.data.DataLoader(dataset=sub_train_tensor,
                                           batch_size=batch_size, 
                                           shuffle=True)

In [85]:
class NetBatchNorm(nn.Module):
    
    # Constructor
    def __init__(self, in_size, n_hidden1, n_hidden2, out_size):
        super(NetBatchNorm, self).__init__()
        self.linear1 = nn.Linear(in_size, n_hidden1)
        self.linear2 = nn.Linear(n_hidden1, n_hidden2)
        self.linear3 = nn.Linear(n_hidden2, out_size)
        self.bn1 = nn.BatchNorm1d(n_hidden1)
        self.bn2 = nn.BatchNorm1d(n_hidden2)
        
    # Prediction
    def forward(self, x):
        x = self.bn1(torch.sigmoid(self.linear1(x)))
        x = self.bn2(torch.sigmoid(self.linear2(x)))
        x = self.linear3(x)
        return x
    
    # Activations, to analyze results 
    def activation(self, x):
        out = []
        z1 = self.bn1(self.linear1(x))
        out.append(z1.detach().numpy().reshape(-1))
        a1 = torch.sigmoid(z1)
        out.append(a1.detach().numpy().reshape(-1).reshape(-1))
        z2 = self.bn2(self.linear2(a1))
        out.append(z2.detach().numpy().reshape(-1))
        a2 = torch.sigmoid(z2)
        out.append(a2.detach().numpy().reshape(-1))
        return out
    
class Net(nn.Module):
    
    # Constructor
    def __init__(self, in_size, n_hidden1, n_hidden2, out_size):

        super(Net, self).__init__()
        self.linear1 = nn.Linear(in_size, n_hidden1)
        self.linear2 = nn.Linear(n_hidden1, n_hidden2)
        self.linear3 = nn.Linear(n_hidden2, out_size)
    
    # Prediction
    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = self.linear3(x)
        return x
    
    # Activations, to analyze results 
    def activation(self, x):
        out = []
        z1 = self.linear1(x)
        out.append(z1.detach().numpy().reshape(-1))
        a1 = torch.sigmoid(z1)
        out.append(a1.detach().numpy().reshape(-1).reshape(-1))
        z2 = self.linear2(a1)
        out.append(z2.detach().numpy().reshape(-1))
        a2 = torch.sigmoid(z2)
        out.append(a2.detach().numpy().reshape(-1))
        return out 
    
    
def train(model, criterion, train_loader, validation_loader, optimizer, epochs=100):
    i = 0
    useful_stuff = {'training_loss':[], 'validation_accuracy':[]}  

    for epoch in range(epochs):
        for i, (x, y) in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            z = model(x.view(-1, 28 * 28))
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            useful_stuff['training_loss'].append(loss.data.item())
            
        correct = 0
        for x, y in validation_loader:
            model.eval()
            yhat = model(x.view(-1, 28 * 28))
            _, label = torch.max(yhat, 1)
            correct += (label == y).sum().item()
            
        accuracy = 100 * (correct / len(validation_dataset))
        useful_stuff['validation_accuracy'].append(accuracy)
    
    return useful_stuff




In [101]:
# Create Data Loader for both train and validating

train_loader = torch.utils.data.DataLoader(dataset=train_tensor, batch_size=2000, shuffle=True)
#validation_loader = torch.utils.data.DataLoader(dataset=val_tensor, batch_size=5000, shuffle=False)


sub_train_loader = torch.utils.data.DataLoader(dataset=sub_train_tensor,
                                           batch_size=2000, 
                                           shuffle=True)

type(train_loader)

torch.utils.data.dataloader.DataLoader

In [99]:
# Create the criterion function
criterion = nn.CrossEntropyLoss

# Set the parameters
input_dim = 12
hidden_dim = 100
output_dim = 10

In [100]:
# Create model, optimizer and train the model

model_norm  = NetBatchNorm(input_dim, hidden_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model_norm.parameters(), lr = 0.1)
#training_results_Norm=train(model_norm , criterion, train_loader, validation_loader, optimizer, epochs=5)

In [79]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [80]:
model = ConvNet(num_classes).to(device)

In [81]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [84]:
# Train the model
total_step = len(sub_train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(sub_train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

ValueError: too many values to unpack (expected 2)